# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [3]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [4]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [5]:
aqdet.LoadDetectors(['taggerNew.json'])

taggerNew.json


Now, to get the data plotted (first 40 channels):

In [15]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)
taggerchan     = Hist(taggerChannels,0,taggerChannels,name='taggerChan')

# histos = [None]*taggerChannels
histos = [None]*40

# for i in range(taggerChannels):
for i in range(40):
    histos[i] = Hist(1200,300,1500)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            # for i in range(taggerChannels):
            for i in range(40):
                if (dat['channel']==(i)):
                    histos[i].Fill(time)
        
    
aq.runFunction(plotCalTagger,0,50000)

ROOT.enableJSVis()

# c1 = ROOT.TCanvas("my1","Title",1000,(taggerChannels/16)*1000)
# c1.Divide(4,taggerChannels/4,0,0)
c1 = ROOT.TCanvas("my1","Title",1000,2500)
c1.Divide(4,10,0,0)

# for i in range(taggerChannels):
for i in range(40):
    c1.cd(i+1)
    histos[i].SetTitle("Channel "+str(i))
    histos[i].Draw("colz")
c1.Draw()


 FCN=287.757 FROM MIGRAD    STATUS=CONVERGED     183 CALLS         184 TOTAL
                     EDM=3.29172e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.90796e+01   1.00904e+01   5.09357e-03  -6.42559e-04
   2  Mean         9.92563e+02   1.18593e+02   2.84605e-02   7.67836e-05
   3  Sigma        1.21038e+02   5.24635e+01   1.69886e-04  -6.27953e-03


In [21]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)
taggerchan     = Hist(taggerChannels,0,taggerChannels,name='taggerChan')

h1 = Hist(1200,300,1500)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            # for i in range(taggerChannels):
            if (dat['channel']==1):
                h1.Fill(time)
        
    
aq.runFunction(plotCalTagger,0,50000)

ROOT.enableJSVis()

c1 = ROOT.TCanvas("my1","Title",1000,1000)

h1.SetTitle("Channel 1")
h1.Draw("colz")
h1.Fit('gaus','','',800,1000)   
print(h1.GetMean(1))
c1.Draw()

919.807044881
 FCN=468.047 FROM MIGRAD    STATUS=CONVERGED     143 CALLS         144 TOTAL
                     EDM=4.13996e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     9.37581e+00   3.56816e-01   2.45654e-03  -4.50415e-05
   2  Mean         8.88237e+02   5.40655e+00   5.48824e-02  -2.53705e-06
   3  Sigma        1.07431e+02   1.20707e+01   3.82913e-04   1.55004e-04
